In [ ]:
import requests
from pw import *
from sqlalchemy import create_engine
from pandas.io.json import json_normalize

FPL_URL = "https://fantasy.premierleague.com/drf/"
TEAM_URL = FPL_URL + "teams/"
FIXTURES_URL = FPL_URL + "fixtures/"
EVENTS_URL = FPL_URL + "events/"
PLAYERS_INFO_SUBURL =  FPL_URL +  "bootstrap-static"

player_dict = {}
response_dict = {}

def getSeason():
    import datetime
    today  = datetime.date.today()
    if today.month >= 8:
        return str(today.year) + "-" +  str(today.year +1)
    else:
        return str(today.year -1) + "_" + str(today.year)

def get_player(id):
    url = "https://fantasy.premierleague.com/drf/element-summary/"
    string_id = str(id)
    return (requests.get(url + string_id)).json()

request_dict = {
  "TEAM_URL": TEAM_URL,
 "FIXTURES_URL": FIXTURES_URL,
    "EVENTS_URL" : EVENTS_URL,
    "PLAYERS_INFO_FILENAME" : PLAYERS_INFO_SUBURL
}

for name,url in request_dict.items():
    response_dict.update({name: requests.get(url)})
    
sched_rslt_df = json_normalize(response_dict['FIXTURES_URL'].json()).drop(columns=['stats'])
players_info_df = json_normalize(response_dict['PLAYERS_INFO_FILENAME'].json()['elements'])
team_df = json_normalize(response_dict['TEAM_URL'].json()).drop(columns=['next_event_fixture','current_event_fixture'])

for _id, _player in players_info_df[['id','web_name']].itertuples(index=False):
    print ("Collecting: " + _player)
    player_dict.update({str(_id): get_player(_id) })
    
for i, p in enumerate(player_dict.keys()):
    if i == 0:
        players_df = json_normalize(player_dict[p]['history'])
    else:
        players_df = players_df.append(json_normalize(player_dict[p]['history']), ignore_index=True)     

        
engine = create_engine("mysql+pymysql://" + usr + ":" + pwrd + "@" + srv +  "/" + db)
con = engine.connect()

players_df['season'] = getSeason()
players_info_df['season']=getSeason()
team_df['season']= getSeason()
sched_rslt_df['season']= getSeason()
sched_rslt_df = json_normalize(response_dict['FIXTURES_URL'].json()).drop(columns=['stats','code','kickoff_time_formatted',
                                                                                   'deadline_time','deadline_time_formatted']) 

players_df.rename(index=str, columns={"id": "player_id"},inplace=True)
players_df.to_sql(name='tbl_plyr_stats_stg',con=con,if_exists='replace', index=False)
players_info_df.to_sql(name='tbl_plyr_info_stg',con=con,if_exists='replace', index=False)
team_df.to_sql(name='tbl_team_info_stg',con=con,if_exists='replace', index=False)
sched_rslt_df.to_sql(name='tbl_fixtures_stg',con=con,if_exists='replace', index=False)

In [2]:
from pw import *
usr

'victor'